<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">Visualización de Datos</p>
<p style="margin: 0; text-align:right;">Máster Universitario de Ciencia de datos </p>
<p style="margin: 0; text-align:right;">Universitat Oberta de Catalunya</p>
    <p style="margin: 0; text-align:right; padding-button: 100px;">Autor: Jorge Alonso Hernández</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# Práctica 2: Creación de la visualización y entrega del proyecto

In [1]:
#Carga de librerías
import pandas as pd
import numpy as np 
import itertools

from datetime import datetime

In [2]:
data_covid = pd.read_csv("datasets/covid_canarias.csv")
municipios = pd.read_csv("datasets/municipios.csv")
islas = pd.read_csv("datasets/islas.csv")

In [3]:
data_covid.head()

,fecha_datos,isla,municipio,sexo,grupo_edad,fecha_caso,fecha_fallecido,fecha_curado,estado_caso,identificacion_caso,fallecido,curado,continua_enfermo,provincia
0,14/03/2022,TENERIFE,Adeje,Hombre,30 - 39,02/01/2022,NaN,13/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife
1,14/03/2022,TENERIFE,Adeje,Hombre,30 - 39,08/01/2022,NaN,19/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife
2,14/03/2022,TENERIFE,Adeje,Hombre,40 - 49,06/01/2022,NaN,17/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife
3,14/03/2022,TENERIFE,Adeje,Hombre,50 - 59,02/01/2022,NaN,13/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife
4,14/03/2022,TENERIFE,Adeje,Hombre,40 - 49,08/01/2022,NaN,19/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife


In [4]:
asignacion = data_covid[data_covid['identificacion_caso'] == 'Asignación']
residencia = data_covid[data_covid['identificacion_caso'] == 'Residencia']

In [5]:
df_municipios = pd.DataFrame(municipios['municipio'])
df_islas = pd.DataFrame(islas['isla'])
df_provincias = pd.DataFrame(pd.unique(islas['provincia']))
df_provincias.columns = ['provincia']

In [167]:
def conversionV2(df, tipo_agrupamiento):
    df_result = pd.DataFrame()
    df_result_accumul = pd.DataFrame()
        
    if tipo_agrupamiento == 'municipios':
        df_agrupamientos = df_municipios
        label = 'municipio'
    elif tipo_agrupamiento == 'islas':
        df_agrupamientos = df_islas
        label = 'isla'
    elif tipo_agrupamiento == 'provincias':
        df_agrupamientos = df_provincias
        label = 'provincia'
    else:
        return None
    
    for index, row in df_agrupamientos.iterrows():
        aux = df[df[label] == row[label]]
        aux = aux.groupby(['fecha_caso',label]).size().reset_index()
        aux.columns=['fecha', label, row[label]]
        aux = aux[['fecha', row[label]]]
        aux['fecha'] = pd.to_datetime(aux['fecha'], dayfirst=True,format='%d/%m/%Y', exact=True)
        aux['fecha'] = aux['fecha'].dt.strftime('%Y/%m/%d')
        
        aux = aux.sort_values(by='fecha')
        aux = aux.set_index('fecha')
        aux_acc = aux.cumsum()
        aux = aux.transpose()
        aux_acc = aux_acc.transpose()
        
        df_result = pd.concat([df_result, aux], axis = 0)
        df_result_accumul = pd.concat([df_result_accumul, aux_acc], axis = 0)
    
    
    ## Ordenamos por fecha y cumplimentamos los NaN con el valor anterior
    df_result_accumul = df_result_accumul.transpose()
    df_result_accumul = df_result_accumul.sort_values(by=['fecha'])
    df_result_accumul[0:1] = df_result_accumul[0:1].fillna(0)   
    df_result_accumul = df_result_accumul.fillna(method="pad")
    df_result_accumul = df_result_accumul.transpose()
    
    #Ordenamos por fecha y cumplimentamos los NaN por 0
    df_result = df_result.replace(np.nan, 0)
    df_result = df_result.transpose()
    df_result = df_result.sort_values(by=['fecha'])
    df_result = df_result.transpose()
    
    df_result = df_result.reset_index()
    df_result_accumul = df_result_accumul.reset_index()
    
    return df_result, df_result_accumul

In [168]:
data_municipios, data_municipios_accumul =  conversionV2(asignacion, 'municipios')

In [169]:
data_islas, data_islas_accumul = conversionV2(asignacion, 'islas')

In [170]:
data_provincias, data_provincias_accumul = conversionV2(asignacion, 'provincias')

In [171]:
print(data_islas_accumul)

fecha          index  2021/01/01  2021/01/02  2021/01/03  2021/01/04  \
0          EL HIERRO         0.0         0.0         1.0        14.0   
1      FUERTEVENTURA         1.0         5.0        17.0        21.0   
2       GRAN CANARIA        62.0       125.0       208.0       280.0   
3          LA GOMERA         0.0         1.0         1.0         5.0   
4           LA PALMA         0.0         3.0         6.0         7.0   
5          LANZAROTE        14.0        46.0        66.0        87.0   
6           TENERIFE        67.0       150.0       199.0       315.0   

fecha  2021/01/05  2021/01/06  2021/01/07  2021/01/08  2021/01/09  ...  \
0            15.0        15.0        16.0        18.0        18.0  ...   
1            47.0        63.0        78.0       107.0       137.0  ...   
2           379.0       458.0       533.0       681.0       790.0  ...   
3             5.0         5.0         5.0         8.0        13.0  ...   
4            11.0        11.0        13.0        22.0

In [172]:
data_provincias.to_excel('data/provincias_asignacion.xlsx', index=False, encoding="utf8")
data_provincias_accumul.to_excel('data/provincias_asignacion_acc.xlsx', index=False, encoding="utf8")
data_islas.to_excel('data/islas_asignacion.xlsx', index=False, encoding="utf8")
data_islas_accumul.to_excel('data/islas_asignacion_acc.xlsx', index=False, encoding="utf8")
data_municipios.to_excel('data/municipios_asignacion.xlsx', index=False, encoding="utf8")
data_municipios_accumul.to_excel('data/municipios_asignacion_acc.xlsx', index=False, encoding="utf8")